In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

#!pip install beautifulsoup4
#from bs4 import BeautifulSoup
#import requests

print('Libraries imported.')

Libraries imported.


In [2]:
!wget -q -O 'toronto_data.json' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
print('Data downloaded!')

Data downloaded!


In [3]:
# Link To Extract
path='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
# Read File
df_wiki=pd.read_html(path)
#Check the type
type(df_wiki)
# Call the position where the table is stored
neighborhood=df_wiki[0]
# Rename the Columns
neighborhood.rename(columns={0:'Postcode', 1: 'Borough', 2: 'Neighborhood'}, inplace=True)
# Eliminate the first row
neighborhood=neighborhood.drop([0])
# Eliminate "Not assigned", categorical values from "Borough" Column
neighborhood=neighborhood[neighborhood.Borough !='Not assigned']
# Making DataFrame
neighborhood=pd.DataFrame(neighborhood)
neighborhood.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [4]:
# Merging rows with same Postcode
neighborhood.set_index(['Postal Code','Borough'],inplace=True)
neighborhood.head()

,,Neighbourhood
Postal Code,Borough,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [5]:
merge_result = neighborhood.groupby(level=['Postal Code','Borough'], sort=False).agg( ','.join)
# Setting the index
serial_wise=merge_result.reset_index()

In [6]:
# Assign the 'Borough' column value to 'Neighborhood' where 'Not assigned' occurs
serial_wise.loc[4, 'Neighborhood']='Queen\'s Park'
# Saving the file for future use!
#erial_wise.to_excel('wikipedia_table.xls')
# Showing the Data Frame
df=pd.DataFrame(serial_wise)
df.head()

,Postal Code,Borough,Neighbourhood,Neighborhood
0,M3A,North York,Parkwoods,NaN
1,M4A,North York,Victoria Village,NaN
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",NaN
3,M6A,North York,"Lawrence Manor, Lawrence Heights",NaN
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",Queen's Park


In [7]:
df.drop('Neighborhood',axis=1,inplace=True)
df.head(12)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [9]:
df1=pd.read_csv("http://cocl.us/Geospatial_data")
#Cancatenation
frames=[df,df1]
frames=pd.concat(frames, axis=1, sort=False)
frames.head()
# Merging the two columns on 'Postcode'
merge_columns=pd.merge(df, df1, left_on='Postal Code', right_on='Postal Code')

In [10]:
merge_columns.to_csv('neigbors_geographical.csv')
merge_columns.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
